In [1]:
import pandas as pd
import os
import numpy as np
import geopandas as gpd
import geoplot.crs as gcrs
import geoplot as gplt
import seaborn as sns
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import sys
%matplotlib inline 

In [2]:
path = os.getcwd()
files = os.listdir(path)
files

['.ipynb_checkpoints',
 'AISDetector.ipynb',
 'bayarea_allwater-fgdc.xml',
 'bayarea_allwater-iso19110.xml',
 'bayarea_allwater-iso19139.xml',
 'bayarea_allwater.dbf',
 'bayarea_allwater.prj',
 'bayarea_allwater.sbn',
 'bayarea_allwater.sbx',
 'bayarea_allwater.shp',
 'bayarea_allwater.shp.xml',
 'bayarea_allwater.shx',
 'ca-county-boundaries.shapefile',
 'CA_Counties_TIGER2016.cpg',
 'CA_Counties_TIGER2016.dbf',
 'CA_Counties_TIGER2016.prj',
 'CA_Counties_TIGER2016.sbn',
 'CA_Counties_TIGER2016.sbx',
 'CA_Counties_TIGER2016.shp',
 'CA_Counties_TIGER2016.shp.xml',
 'CA_Counties_TIGER2016.shx',
 'CTPPROAD-fgdc.xml',
 'CTPPROAD-iso19110.xml',
 'CTPPROAD-iso19139.xml',
 'CTPPROAD.dbf',
 'CTPPROAD.prj',
 'CTPPROAD.sbn',
 'CTPPROAD.sbx',
 'CTPPROAD.shp',
 'CTPPROAD.shp.xml',
 'CTPPROAD.shx',
 'data.zip',
 'DataBase',
 'geopandas',
 'GIS_ADMIN_OCEAN_BAY.dbf',
 'GIS_ADMIN_OCEAN_BAY.prj',
 'GIS_ADMIN_OCEAN_BAY.sbn',
 'GIS_ADMIN_OCEAN_BAY.sbx',
 'GIS_ADMIN_OCEAN_BAY.shp',
 'GIS_ADMIN_OCEAN_BAY.

In [3]:
files_csv = [f for f in files if f[-4:] == 'xlsx']
files_csv

['WorkOrder8180645_NAIS_2018-01.xlsx', 'WorkOrder8180645_NAIS_2018-02.xlsx']

In [ ]:
#df = pd.DataFrame()
#for f in files_csv:
#    data = pd.read_excel(f, sort=True)
#    df = df.append(data)

In [ ]:
#df

In [ ]:
#------------------------------------------------------------------------------------

In [4]:
San_fran = gdp.read_file('bayarea_allwater.shp')
San_fran

NameError: name 'gdp' is not defined

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
San_fran.plot(ax=ax, alpha =1, color = 'grey')

In [ ]:
ship = pd.read_excel('WorkOrder8180645_NAIS_2018-01.xlsx')

In [ ]:
ship

In [ ]:
crs = {'init': 'espg:4326'}
geometry = [Point(xy) for xy in zip(ship['LON_AVG'], ship['LAT_AVG'])]

geometry[:5]

In [ ]:
geo_ship = gpd.GeoDataFrame(ship, crs=crs, geometry=geometry)
geo_ship.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
San_fran.plot(ax=ax, alpha=1, color = 'grey')
geo_ship.plot(ax=ax, markersize = 10, color ='blue', marker= 'o', label= 'lat/lon')

In [ ]:
ship.info()

In [ ]:
ship.groupby("DIM_").size()
#ship["CALL_SIGN"].unique()
#ship["NAME"].unique()

In [ ]:
#ship.groupby('CALL_SIGN').size()
ship.groupby('DRAUGHT').size()

In [ ]:
plt.scatter(ship['LAT_AVG'], ship['LON_AVG'])

In [ ]:
Dft1 = ship[ship['DRAUGHT'] == '1.9']
Dft9 = ship[ship['DRAUGHT'] == '9.0']
fig = plt.figure(figsize = (8,6))
ax1 = fig.add_subplot(111)

ax1.scatter(Dft1['LAT_AVG'], Dft1['LON_AVG'], c='b', marker=".", label='Draft 1.9')
ax1.scatter(Dft9['LAT_AVG'], Dft9['LON_AVG'], c='r', marker=".", label='Draft 9')
plt.legend(loc='lower left');
plt.show()

In [ ]:
Dft9.groupby('CALL_SIGN').size()

In [ ]:
ship["DAY"] = ship['PERIOD'].dt.date
ship.head()

In [ ]:
ship['PERIOD'] = pd.to_datetime(ship['PERIOD'])
#ship.tail(100)
ship.info()

In [ ]:
def extract_feature(ship):
    distance = np.sqrt(((ship['LAT_AVG'].diff())**2) + ((ship['LON_AVG'].diff())**2))
    interval = ship['PERIOD'].diff().astype(np.int64)/1e9
    velocity = distance/interval
    Vel_mean = velocity.mean()
    Vel_stdv = velocity.std()
    return pd.Series({'Vm' :Vel_mean, 'Vs': Vel_stdv})

In [ ]:
ship.groupby(["MMSI", "DAY"]).apply(extract_feature)

In [ ]:
############################
############################
############################
############################

In [ ]:
#preprocessing Data
bins = (-0.5, 0.5, 1.5)
group_names = ['bad', 'good']
ship['legitimate'] = pd.cut(Mal['legitimate'], bins = bins, labels = group_names)
ship['legitimate'].unique() 

In [ ]:
#differentiating good=1 and bad=0
label_InitialSize = LabelEncoder() 

In [ ]:
Mal['legitimate'] = label_InitialSize.fit_transform(Mal['legitimate'])

In [ ]:
Mal['legitimate'].value_counts()

In [ ]:
#sns.countplot(Mal['legitimate'])

In [ ]:
#Seperate our data set as response variable and feature variables
#legitimate is the feature that will predict the quality
X = Mal.drop('legitimate', axis = 1) #predict the quality 
y = Mal['legitimate'] #quality itself

In [ ]:
#train and test splitting of data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 31)

In [ ]:
len(X_train)

In [ ]:
#to get optimized results
#values are huge, biased results
#scaler fixed that
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
X_train[:2]

# SVM Classifier

In [ ]:
#Support Vector Model
#better on smaller number
#Faster and easiest to apply model on
clf = svm.SVC()
clf.fit(X_train, y_train)
pred_clf = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, pred_clf))
print(confusion_matrix(y_test, pred_clf))

# Random Forest

In [ ]:
#best for medium size data set
rfc = RandomForestClassifier(n_estimators=500)
rfc.fit(X_train, y_train)
pred_rfc = rfc.predict(X_test)

In [ ]:
#X_test[:20]

In [ ]:
print(classification_report(y_test, pred_rfc))
print(confusion_matrix(y_test, pred_rfc))

# Neural Network

In [ ]:
#deep learning
#works best on large dataset and text data
#hidden laers = nodes
mlpc = MLPClassifier(hidden_layer_sizes=(11,11,11), max_iter = 500)
mlpc.fit(X_train, y_train)
pred_mlpc = mlpc.predict(X_test)

In [ ]:
print(classification_report(y_test, pred_mlpc))
print(confusion_matrix(y_test, pred_mlpc))

In [ ]:
Mal.head()

In [ ]:
XNewPre = [[332, 224, 258, 9, 0, 361984, 115712, 0, 6135, 4096, 372736, 4194304, 4096, 512, 0, 0, 1036288, 1024, 485887, 16, 1024, 1048576, 4096, 1048576, 4096, 0, 16]]
ANewPre = [[332,224,3330,9,0,130560,19968,0,81778,4096,143360,771751936,4096,512,5,1,159744,1024,188943,2,33088,1048576,4096,1048576,4096,0,16
]]
DNewPre = [[332,224,33166,2,25,60928,55296,0,13848,4096,65536,4194304,4096,512,4,0,135168,1024,0,2,0,1048576,16384,1048576,4096,0,16
]]
XNewPre = sc.transform(XNewPre)
ANewPre = sc.transform(ANewPre)
DnewPre = sc.transform(DNewPre)
BNewPre = rfc.predict(XNewPre)
YNewPre = rfc.predict(ANewPre)
QNewPre = rfc.predict(DNewPre)
print(YNewPre, BNewPre, QNewPre)

In [ ]:
NewMal = pd.read_csv('final.csv')

In [ ]:
NewMal.head() 

In [ ]:
k=[]
for i in range(0, len(NewMal)):
    k.append([])
    k[i].append(NewMal.loc[i][0])

NewMal = NewMal.drop('Name', axis=1)
NewMal = NewMal.drop('md5', axis=1)
NewMal = NewMal.drop('legitimate', axis=1)
NewMal = NewMal.drop('Win32VersionValue', axis=1)
NewMal.head()

In [ ]:
X = []
Y = []
for i in range(0, len(NewMal)):
    X.append([])
    for x in NewMal:
        X[i].append(NewMal.loc[i][x])
    X[i]=[X[i]]
    
    Y = sc.transform(X[i])      #scalar transform before running
    DNewPre = rfc.predict(Y)    #prediction value 1
    print(k[i], 'is', DNewPre)

        

In [ ]:
print(NewMal.loc[1][0])